## 06. 이미지 회전과 원근 변환

In [18]:
import cv2 as cv
import numpy as np

CHA_PATH = "../images/cha.jpg"
CARD_PATH = "../images/card.jpg"
SEO_PATH = "../images/seo.jpg"

### 6-1. 이미지 회전 (rotate)
- `cv2.rotate(img, rotateCode)`
- `rotateCode`
    - `cv2.ROTATE_90_CLOCKWISE` : 시계방향 90도 회전
    - `cv2.ROTATE_180` : 180도 회전
    - `cv2.ROTATE_90_COUNTERCLOCKWISE` : 반시계방향 90도 회전

In [2]:
img = cv.imread(CHA_PATH)

dst_90 = cv.rotate(img, cv.ROTATE_90_CLOCKWISE)
dst_180 = cv.rotate(img, cv.ROTATE_180)
dst_90_counter = cv.rotate(img, cv.ROTATE_90_COUNTERCLOCKWISE)

cv.imshow("original", img)
cv.imshow("90", dst_90)
cv.imshow("180", dst_180)
cv.imshow("90_counter", dst_90_counter)

cv.waitKey(0)
cv.destroyAllWindows()

### 6-2. 이미지 회전 (아핀 변환)
- 2D 또는 3D 공간에서 이미지를 이동, 회전, 크기 조정, 반사, 기울이기와 같은 변환을 수학적으로 처리하는 방법
- 이를 수행하는 공식은 행렬 연산을 사용하여 정의됨

In [ ]:
# 아핀 변환(math 모듈로 직접 구현)
import math

img = cv.imread(CHA_PATH)

rad = 45 * math.pi / 180
affine = np.array([[math.cos(rad), -math.sin(rad), 0],
                   [math.sin(rad), math.cos(rad), 0]])

dst = cv.warpAffine(img, affine, (img.shape[1], img.shape[0]))

cv.imshow("img", img)
cv.imshow("rotate", dst)

cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
# 아핀 변환(getRotationMatrix2D 이용)
img = cv.imread(CHA_PATH)

center = (int(img.shape[1]/2), int(img.shape[0]/2))
scale = (img.shape[1], img.shape[0])
affine = cv.getRotationMatrix2D(center, 45, 1)
dst = cv.warpAffine(img, affine, scale)

cv.imshow("img", img)
cv.imshow("rotate", dst)

cv.waitKey(0)
cv.destroyAllWindows()


In [ ]:
# 트랙바와 함께 사용
img = cv.imread(CHA_PATH)

name = "Rotation"
cv.namedWindow(name)

trackbar_name = "angle"
cv.createTrackbar(trackbar_name, name, 0, 360, lambda x:x)

while True:
    angle = cv.getTrackbarPos(trackbar_name, name)
    center = (int(img.shape[1]/2), int(img.shape[0]/2))
    scale = (img.shape[1], img.shape[0])
    affine = cv.getRotationMatrix2D(center, angle, 0.5)
    dst = cv.warpAffine(img, affine, scale)

    cv.imshow(name, dst)

    if cv.waitKey(1) == ord("q"):
        break

cv.destroyAllWindows()

### 6-3. 원근 변환
- `cv2.getPerspectiveTransform(src, dst) -> mat`
- `cv2.warpPerspective(img, mat, (width, height))`

In [18]:
img = cv.imread(CARD_PATH)

width, height = 600, 350

# 원근을 변형할 지점 선택 : 시계방향
src = np.array([[54,261], [981, 128], [1213, 560], [194, 735]], dtype=np.float32)

# 결과물을 출력할 형태 : 시계방향
dst = np.array([[0,0], [width, 0], [width, height], [0, height]], dtype=np.float32)

# 원근 변형하기 위한 Matrix 생성
mat = cv.getPerspectiveTransform(src, dst)
# print(mat)

# img를 mat에 의해 변환
result = cv.warpPerspective(img, mat, (width, height))

cv.imshow("img", img)
cv.imshow("warped", result)

cv.waitKey(0)
cv.destroyAllWindows()

In [14]:
# 실습. 이미지 원근 변환하기

img = cv.imread('../images/dollar.jpg')

width, height = 500, 200

# 원근을 변형할 지점 선택 : 시계방향
src = np.array([[285,123], [694, 470], [386, 619], [40, 202]], dtype=np.float32)

# 결과물을 출력할 형태 : 시계방향
dst = np.array([[0,0], [width, 0], [width, height], [0, height]], dtype=np.float32)

# 원근 변형하기 위한 Matrix 생성
mat = cv.getPerspectiveTransform(src, dst)
# print(mat)

# img를 mat에 의해 변환
result = cv.warpPerspective(img, mat, (width, height))

cv.imshow("img", img)
cv.imshow("warped", result)

cv.waitKey(0)
cv.destroyAllWindows()

### 6-4. 마우스 이벤트
- `cv2.setMouseCallback(windowName, onMouse)`
    - `onMouse(event)` : 마우스 이벤트를 처리하는 콜백함수. 첫번째 인자로 이벤트를 받아줌.

In [25]:
img = cv.imread(SEO_PATH)
name = "Mouse Event"
cv.namedWindow(name)

def on_mouse(event, x, y, flags, _):
    # print(event, x, y, flags)
    if event == cv.EVENT_LBUTTONDOWN:
        copied = img.copy()
        cv.circle(copied, (x,y), 20, (200, 200, 255), cv.FILLED)
        cv.imshow(name, copied)

cv.setMouseCallback(name, on_mouse)
cv.imshow(name, img)
cv.waitKey(0)
cv.destroyAllWindows()

In [62]:
# 실습. 반자동 문서 스캐너 만들기(내 풀이)
img = cv.imread('../images/dollar.jpg')
img_copy = img.copy()
points = []
width, height = 500, 200
name = "Scanner"
cv.namedWindow(name)

def on_mouse(event, x, y, flags, _):

    if event == cv.EVENT_LBUTTONDOWN and len(points) < 4:
        points.append([x, y])       
        # 점
        copied = img_copy.copy()
        for i, point in enumerate(points):
            cv.circle(copied, tuple(point), 5, (0, 255, 0), cv.FILLED)        
        # 라인 연결
        for i in range(len(points)-1):
            cv.line(copied, tuple(points[i]), tuple(points[i+1]), (255, 0, 0), 2)
        if len(points) == 4:
            cv.line(copied, tuple(points[3]), tuple(points[0]), (255, 0, 0), 2)
        cv.imshow(name, copied) 
        # 원근 변환       
        if len(points) == 4:
            src = np.array(points, dtype=np.float32)
            dst = np.array([[0,0], [width, 0], [width, height], [0, height]], dtype=np.float32)
            mat = cv.getPerspectiveTransform(src, dst)
            result = cv.warpPerspective(img_copy, mat, (width, height))            
            cv.imshow("Scanned", result)

    # 라인이 마우스를 따라가도록 구현
    elif event == cv.EVENT_MOUSEMOVE and len(points) > 0 and len(points) < 4:
        copied = img_copy.copy()
        for i, point in enumerate(points):
            cv.circle(copied, tuple(point), 8, (0, 255, 0), cv.FILLED)       
        for i in range(len(points)-1):
            cv.line(copied, tuple(points[i]), tuple(points[i+1]), (255, 0, 0), 2)        
        cv.line(copied, tuple(points[-1]), (x, y), (0, 255, 255), 1)                
        cv.imshow(name, copied)

cv.setMouseCallback(name, on_mouse)
cv.imshow(name, img)
while True:
    if cv.waitKey(1) == ord("q"):
        break
cv.destroyAllWindows()

In [ ]:
# 실습. 반자동 문서 스캐너 만들기 (강사님 풀이)
img = cv.imread('../images/dollar.jpg')

name = "Scanner"
point_list = []

# 원근변환을 실행
def show_result():      
  width = int(np.linalg.norm(np.array(point_list[0]) - np.array(point_list[1])))
  height = int(np.linalg.norm(np.array(point_list[0]) - np.array(point_list[3])))

  src = np.array(point_list, dtype=np.float32)
  dst = np.array([[0,0], [width, 0], [width, height], [0, height]], dtype=np.float32)
  mat = cv.getPerspectiveTransform(src, dst)
  result = cv.warpPerspective(img, mat, (width, height))
  cv.imshow("Cropped", result)

# 클릭할 때 클릭한 지점을 리스트에 추가
drawing = False
def on_mouse(event, x, y, flags, _):
  global drawing
  coppied = img.copy()

  if event == cv.EVENT_LBUTTONDOWN:
    drawing = True
    point = (x,y)
    point_list.append(point)

  if drawing:
    prev_point = None
    for p in point_list:
      cv.circle(coppied, p, 10, (0,255,255), cv.FILLED)
      if prev_point:
        cv.line(coppied, prev_point, p, (0,255,255), 5)
      prev_point = p

    next_point = (x,y)
    if len(point_list) == 4:
      next_point = point_list[0]
      show_result()
    cv.line(coppied, point_list[-1], next_point, (0,255,255),5)

  cv.imshow(name, coppied)

cv.namedWindow(name)
cv.setMouseCallback(name, on_mouse)
cv.imshow(name, img)
cv.waitKey(0)
cv.destroyAllWindows()